Nama : Andy Nugraha Putra

NIM : 2241720006  

Kelas : 3E

# Praktikum 1
Bagging dengan RandomForest


In [93]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

**NumPy**

Dipakai buat komputasi numerik, terutama yang melibatkan array multidimensi.

**Pandas**

Berguna buat manipulasi dan analisis data yang berbentuk tabel, biasanya pakai DataFrame.

**Scikit-learn**

- DecisionTreeClassifier: Model untuk klasifikasi yang berbasis pada pohon keputusan.

- RandomForestClassifier: Model yang gabungin beberapa pohon keputusan (ensemble) untuk ningkatin akurasi prediksi.

- train_test_split: Buat bagi dataset jadi dua, data latih (training) dan data uji (testing).

- accuracy_score: Ngukur akurasi model dengan ngecek seberapa pas prediksinya dengan nilai sebenarnya.

- classification_report: Ngasih evaluasi detail, kayak precision, recall, dan F1-score buat setiap kelas.

**Persiapan data**

In [94]:
# Load data
df = pd.read_csv('/content/wbc.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [95]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Penjelasan:

- `df = pd.read_csv('/content/wbc.csv')`: Kode ini digunakan untuk mengimpor dataset dari file **CSV** yang ada di path tersebut. Data dari file akan disimpan dalam variabel `df` dalam bentuk DataFrame, yang mudah diolah menggunakan Pandas.

- `df.head()`: Kode ini digunakan buat menampilkan 5 baris pertama dari dataset dalam bentuk tabel. Ini berguna untuk melihat sekilas isi data dan memeriksa apakah dataset sudah diimpor dengan benar.

In [96]:
# Cek kolom null
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


penjelasan : Fungsi df.isnull() mengidentifikasi nilai yang kosong, dan sum() menjumlahkannya untuk setiap kolom

In [97]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,3:-1]
y = df['diagnosis']
y = y.map({'M':1, 'B':0}) # Encode label

# Cek jumlah fitur dan instance
X.shape

(569, 29)

Penjelasan :

1. **`X = df.iloc[:,3:-1]`**: Memilih kolom fitur dari 'radius_mean' sampai 'fractal_dimension_worst'.
2. **`y = df['diagnosis']`**: Mengambil kolom 'diagnosis' sebagai label.
3. **`y = y.map({'M':1, 'B':0})`**: Mengubah label 'M' jadi 1 (Malignant) dan 'B' jadi 0 (Benign).
4. **`X.shape`**: Mengecek jumlah baris dan kolom dalam fitur (X).

Ini menyiapkan data fitur dan label untuk model machine learning.

**Split data training dan testing**

In [98]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

penjelasan : Kode diastas memisahkan data menjadi set pelatihan dan pengujian menggunakan fungsi train_test_split dari Scikit-learn

**Traning Decision Tree**

In [99]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.95
Test set accuracy: 0.9473684210526315


penjelasan :



1. **`dt = DecisionTreeClassifier()`**: Membuat model **Decision Tree** dengan pengaturan default.
2. **`dt.fit(X_train, y_train)`**: Melatih model dengan data latih.
3. **`y_pred_dt = dt.predict(X_test)`**: Memprediksi label data uji.
4. **`acc_dt = accuracy_score(y_test, y_pred_dt)`**: Menghitung akurasi model.
5. **`print()`**: Menampilkan akurasi pada data uji.

Model ini dilatih dan diuji untuk melihat seberapa baik akurasi prediksinya.

**Training RandomForest**

In [100]:
# Pada kasus kali ini kita akan menggunakan estimator pada RandomForest
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

rf = RandomForestClassifier(n_estimators=10, random_state=1)

# Sesuaikan dt ke set training
rf.fit(X_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


penjelasan :


1. **`rf = RandomForestClassifier(n_estimators=10, random_state=1)`**: Membuat model **Random Forest** dengan 10 pohon keputusan.
2. **`rf.fit(X_train, y_train)`**: Melatih model dengan data latih.
3. **`y_pred_rf = rf.predict(X_test)`**: Memprediksi label data uji.
4. **`acc_rf = accuracy_score(y_test, y_pred_rf)`**: Menghitung akurasi model.
5. **`print()`**: Menampilkan akurasi pada data uji.

Model ini menggunakan Random Forest untuk memprediksi dan mengevaluasi akurasinya.

# Praktikum 2
Boosting dengan AdaBoost

**Import Library**

In [101]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder # Kebutuhan encoding label

Penjelasan :   kode diatas digunakan untuk mengimpor library yang dibutuhkan

**Persiapan Data**

In [102]:
# Load data
df = pd.read_csv('/content/iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Penjelasan :   

df = pd.read_csv('/content/iris.csv') digunakan untuk mengimpor dataset

df.head() digunakan untuk menampilkan dataset dalam bentuk table

In [103]:
# Cek kolom null
df.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


Penjelasan :   Fungsi df.isnull() mengidentifikasi nilai yang kosong, dan sum() menjumlahkannya untuk setiap kolom

In [104]:
# Seleksi fitur
X = df.iloc[:,2:-1]
y = df['Species']

# encode label
ec = LabelEncoder()
y = ec.fit_transform(y)

# Cek jumlah fitur dan instance
print(X.shape)

# Cek label
print(y)

(150, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Penjelasan :   



1. **`X = df.iloc[:,2:-1]`**: Memilih kolom fitur dari DataFrame.
2. **`y = df['Species']`**: Mengambil kolom 'Species' sebagai label.
3. **`ec = LabelEncoder()`** dan **`y = ec.fit_transform(y)`**: Mengubah label 'Species' menjadi angka.
4. **`print(X.shape)`**: Menampilkan jumlah baris dan kolom fitur.
5. **`print(y)`**: Menampilkan label yang sudah diencode.

Kode ini mempersiapkan fitur dan label dalam format numerik untuk proses pemodelan.

**Split data training dan testing**

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Penjelasan :   Kode diatas memisahkan data menjadi set pelatihan dan pengujian menggunakan fungsi train_test_split dari Scikit-learn

**Training Decision Tree**

In [106]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


Penjelasan :



1. **`dt = DecisionTreeClassifier()`**: Membuat model **Decision Tree** dengan parameter default.
2. **`dt.fit(X_train, y_train)`**: Melatih model dengan data latih.
3. **`y_pred_dt = dt.predict(X_test)`**: Memprediksi label data uji.
4. **`acc_dt = accuracy_score(y_test, y_pred_dt)`**: Menghitung akurasi model.
5. **`print()`**: Menampilkan akurasi pada data uji.

Kode ini melatih dan menguji model **Decision Tree**, serta mengevaluasi akurasinya.

**Training AdaBoost**

In [107]:
# Pada kasus kali ini kita akan menggunakan estimator pada AdaBoost
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

ada = AdaBoostClassifier(n_estimators=2)

# Sesuaikan dt ke set training
ada.fit(X_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(X_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Penjelasan :  



1. **`ada = AdaBoostClassifier(n_estimators=2)`**: Membuat model **AdaBoost** dengan 2 estimators.
2. **`ada.fit(X_train, y_train)`**: Melatih model menggunakan data latih.
3. **`y_pred_ada = ada.predict(X_test)`**: Memprediksi label data uji.
4. **`acc_ada = accuracy_score(y_test, y_pred_ada)`**: Menghitung akurasi model.
5. **`print()`**: Menampilkan akurasi pada data uji.

Kode ini melatih dan menguji model **AdaBoost**, serta mengevaluasi akurasinya.

# Praktikum 3
Stacking

Lengkapi bagian berikut dengan data sesuai tugas, dan tentukan perbedaan nilai akurasi antara Random Forest, Adaboost, dan Stacking

In [108]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier



layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())


clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.868421052631579

Penjelasan :  


1. **Impor Library**: Mengimpor model dan fungsi dari scikit-learn.
2. **Layer Satu Estimators**: Menggunakan **Random Forest** dan **K-Nearest Neighbors**.
3. **Layer Dua Estimators**: Menggunakan **Decision Tree** dan **Random Forest**, dengan **Logistic Regression** sebagai estimator akhir.
4. **Pembagian Data**: Menggunakan **`train_test_split`** untuk membagi dataset menjadi data latih dan uji.
5. **Melatih dan Menghitung Akurasi**: Melatih model dengan **`fit`** dan menghitung akurasi pada data uji dengan **`score`**.

Model stacking ini menggabungkan beberapa algoritma untuk meningkatkan akurasi prediksi.

# Praktikum 4
Stacking dengan Voting

**Import Library**

In [109]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB # import Naive Bayes model Gaussian (asumsi data terdistribusi normal)
from sklearn.svm import SVC # import SVM classifier
from sklearn.ensemble import VotingClassifier # import model Voting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Penjelasan :  kode diatas digunakan untuk mengimpor library yang dibutuhkan

**Persiapan Data**

In [110]:
# Load Data

dbt = pd.read_csv('/content/diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Penjelasan :   

dbt = pd.read_csv('/content/diabetes.csv') digunakan untuk meload dataset

dbt.head() digunakan untuk memanpilkan dataset dalam bentuk tabel

In [111]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

Penjelasan :  Kode ini digunakan untuk memeriksa nama-nama kolom dalam objek DataFrame dbt. Dengan menggunakan atribut columns

In [112]:
# Cek kolom null
dbt.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


Penjelasan :   Fungsi df.isnull() mengidentifikasi nilai yang kosong, dan sum() menjumlahkannya untuk setiap kolom

In [113]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


Penjelasan : Kode diatas digunakan untuk memeriksa dan mencetak jumlah nilai 0 dalam beberapa kolom yang relevan dari DataFrame dbt

**Daftar Kolom**

feature_columns menyimpan nama-nama kolom yang ingin diperiksa.

**Looping**

Dengan menggunakan loop for, kode ini iterasi melalui setiap kolom dalam feature_columns.

**Mencetak Hasil**

Untuk setiap kolom, kode mencetak garis pemisah dan jumlah nilai 0 yang ditemukan di kolom tersebut dengan len(dbt.loc[dbt[column] == 0]).

In [114]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

Penjelasan :  Kode diatas digunakan untuk menggantikan nilai 0 dalam kolom tertentu dari DataFrame dbt dengan nilai rata-rata (mean) dari kolom tersebut menggunakan SimpleImputer dari Scikit-learn

**Inisialisasi Imputer**

SimpleImputer(missing_values=0, strategy="mean", copy=False) membuat objek fill_values yang akan mengganti nilai 0 dengan nilai rata-rata kolom.

**Imputasi Data**

fill_values.fit_transform(dbt[feature_columns]) menerapkan imputasi pada kolom yang ditentukan dalam feature_columns di DataFrame dbt.

**Mengganti Nilai**

Hasil imputasi kemudian menggantikan nilai asli di kolom-kolom tersebut di DataFrame dbt.

**Split data training dan testing**

In [115]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Penjelasan : kode diatas digunakan untuk mempersiapkan data untuk pelatihan dan pengujian model machine learning dengan membagi dataset dbt menjadi fitur dan target. Kolom yang ditentukan dalam feature_columns digunakan sebagai fitur (X), sedangkan kolom Outcome diambil sebagai label target (y). Kemudian, fungsi train_test_split digunakan untuk membagi data, di mana 30% digunakan untuk pengujian dan 70% untuk pelatihan, dengan parameter random_state=42 yang memastikan pemisahan data dapat direproduksi. Hasilnya adalah empat variabel: X_train, X_test, y_train, dan y_test, yang siap digunakan dalam proses pelatihan dan evaluasi model.

**Training dengan GaussianNB**

**Standarisasi Fitur**

In [116]:
# Karena asumsi Gaussian NB adalah data terdistribusi secara normal,
# maka kita perlu melakukan standarisasi

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

Penjelasan :



1. **Impor Library**: Mengimpor **StandardScaler** dari scikit-learn untuk melakukan standarisasi data.
   
2. **`sc = StandardScaler()`**: Membuat instance dari **StandardScaler**.

3. **Standarisasi Data**:
   - **`X_train_std = sc.fit_transform(X_train)`**: Melakukan standarisasi pada data latih (`X_train`) dan menyimpannya dalam `X_train_std`. Ini menghitung mean dan standar deviasi dari data latih dan menerapkan transformasi.
   - **`X_test_std = sc.transform(X_test)`**: Menerapkan transformasi yang sama ke data uji (`X_test`) tanpa menghitung ulang mean dan standar deviasi, dan menyimpannya dalam `X_test_std`.

Proses ini memastikan bahwa fitur dalam data terdistribusi normal, yang penting untuk model **Gaussian Naive Bayes**.

**Training dan Evaluasi**

In [117]:
# Buat obyek GaussianNB
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data test
y_pred_gnb = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_gnb = accuracy_score(y_test, y_pred_gnb)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_gnb))
print(f"Test set accuracy: {acc_gnb}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


Penjelasan :


1. **Buat Model GaussianNB**: **`gnb_std = GaussianNB()`**: Membuat instance dari model **Gaussian Naive Bayes**.

2. **Melatih Model**: **`gnb_std.fit(X_train_std, y_train)`**: Melatih model dengan data yang sudah distandarisasi (`X_train_std` dan `y_train`).

3. **Prediksi**: **`y_pred_gnb = gnb_std.predict(X_test_std)`**: Menggunakan model yang telah dilatih untuk memprediksi label pada data uji yang sudah distandarisasi (`X_test_std`).

4. **Evaluasi Akurasi**: **`acc_gnb = accuracy_score(y_test, y_pred_gnb)`**: Menghitung akurasi model dengan membandingkan prediksi (`y_pred_gnb`) dengan label asli (`y_test`).

5. **Menampilkan Hasil**:
   - **`print()`**: Menampilkan akurasi model pada data uji dalam format desimal dengan dua angka di belakang koma.

Kode ini melatih dan menguji model **Gaussian Naive Bayes** menggunakan data yang telah distandarisasi dan mengevaluasi akurasinya.

**Training dengan SVM Linier**

In [118]:
# Model SVM linier tanpa tunnning hyperparameter
svm_lin = SVC(kernel='linear')

# Fit ke model
svm_lin.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_lin = svm_lin.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_lin))
print(f"Test set accuracy: {acc_svm_lin}")

Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


Penjelasan :


1. **Buat Model SVM**: **`svm_lin = SVC(kernel='linear')`**: Membuat instance dari model **Support Vector Machine (SVM)** dengan kernel linear.

2. **Melatih Model**: **`svm_lin.fit(X_train_std, y_train)`**: Melatih model dengan data yang sudah distandarisasi (`X_train_std` dan `y_train`).

3. **Prediksi**: **`y_pred_svm_lin = svm_lin.predict(X_test_std)`**: Menggunakan model yang telah dilatih untuk memprediksi label pada data uji yang sudah distandarisasi (`X_test_std`).

4. **Evaluasi Akurasi**: **`acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin)`**: Menghitung akurasi model dengan membandingkan prediksi (`y_pred_svm_lin`) dengan label asli (`y_test`).

5. **Menampilkan Hasil**:
   - **`print()`**: Menampilkan akurasi model pada data uji dalam format desimal dengan dua angka di belakang koma.

Kode ini melatih dan menguji model SVM dengan kernel linear dan mengevaluasi akurasinya.

**Training dengan SVM RBF**

In [119]:
# Model SVM RBF tanpa tunnning hyperparameter
svm_rbf = SVC(kernel='rbf')

# Fit ke model
svm_rbf.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_rbf = svm_rbf.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_rbf))
print(f"Test set accuracy: {acc_svm_rbf}")

Test set accuracy: 0.72
Test set accuracy: 0.7229437229437229


Penjelasan :


1. **Model SVM dengan Kernel RBF**: Membuat instance **SVC** dengan kernel RBF.
2. **Melatih Model**: Menggunakan data yang sudah distandarisasi untuk melatih model.
3. **Prediksi**: Memprediksi label pada data uji yang telah distandarisasi.
4. **Evaluasi Akurasi**: Menghitung akurasi dengan membandingkan prediksi dan label asli.
5. **Menampilkan Hasil**: Mencetak akurasi model pada data uji.

Model SVM ini dilatih dan diuji menggunakan kernel RBF.

**Training dengan Voting**

In [120]:
# Definisikan algoritma yang akan digunakan untuk voting

clf1 = GaussianNB()
clf2 = SVC(kernel='linear')
clf3 = SVC(kernel='rbf', probability=True)

# model hard voting
voting = VotingClassifier(estimators=[('GaussianNB', clf1), ('SVM-LIN', clf2), ('SVM-RBF', clf3)], voting='hard')

# Fit model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_vt1 = voting.predict(X_test_std)

# Evaluasi akurasi testing data
acc_vt1 = accuracy_score(y_test, y_pred_vt1)

# Print hasil evaluasi
print('Voting Hard')
print("Test set accuracy: {:.2f}".format(acc_vt1))
print(f"Test set accuracy: {acc_vt1}")

Voting Hard
Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


Penjelasan :

1. **Definisikan Algoritma**: Membuat model **Gaussian Naive Bayes**, SVM dengan kernel linear, dan SVM dengan kernel RBF (probabilitas diaktifkan).
2. **Model Hard Voting**: Menggabungkan ketiga model dalam **VotingClassifier** dengan voting keras.
3. **Melatih Model**: Melatih model voting dengan data yang distandarisasi.
4. **Prediksi**: Memprediksi label pada data uji.
5. **Evaluasi Akurasi**: Menghitung akurasi dengan membandingkan prediksi dan label asli.
6. **Menampilkan Hasil**: Mencetak akurasi model voting pada data uji.

Model ini menggunakan metode hard voting untuk menggabungkan prediksi dari beberapa algoritma.

# Tugas 1

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

Impor library

In [121]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

Persiapan data

In [122]:
# Load data
df = pd.read_csv('/content/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [123]:
# Cek kolom null
df.isnull().sum()

,0
class,0
cap-shape,0
cap-surface,0
cap-color,0
bruises,0
odor,0
gill-attachment,0
gill-spacing,0
gill-size,0
gill-color,0


In [124]:
#seleksi fitur
X = df.iloc[:,1:]
y = df.iloc[:,0]
X

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,s,o,o,p,o,o,p,b,c,l
8120,x,s,n,f,n,a,c,b,y,e,...,s,o,o,p,n,o,p,b,v,l
8121,f,s,n,f,n,a,c,b,n,e,...,s,o,o,p,o,o,p,b,c,l
8122,k,y,n,f,y,f,c,n,b,t,...,k,w,w,p,w,o,e,w,v,l


encode lebel

In [125]:
from sklearn.preprocessing import LabelEncoder

# Membuat objek LabelEncoder
le = LabelEncoder()

# Mengencode fitur
X_encoded = X.apply(le.fit_transform)

# Mengencode target
y_encoded = le.fit_transform(y)

# Menampilkan hasil encoding
print(X_encoded.head())
print(y_encoded[:5])

   cap-shape  cap-surface  cap-color  bruises  odor  gill-attachment  \
0          5            2          4        1     6                1   
1          5            2          9        1     0                1   
2          0            2          8        1     3                1   
3          5            3          8        1     6                1   
4          5            2          3        0     5                1   

   gill-spacing  gill-size  gill-color  stalk-shape  ...  \
0             0          1           4            0  ...   
1             0          0           4            0  ...   
2             0          0           5            0  ...   
3             0          1           5            0  ...   
4             1          0           4            1  ...   

   stalk-surface-below-ring  stalk-color-above-ring  stalk-color-below-ring  \
0                         2                       7                       7   
1                         2                     

Split data training dan testing

In [126]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

Standarisasi fitur

In [127]:
# Standarisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Training dengan decision tree

In [128]:
dt = DecisionTreeClassifier()

# Tentukan grid hyperparameter yang ingin dituning
param_grid = {
    'max_depth': [None, 5, 10, 15, 20],  # Maksimal kedalaman tree
    'min_samples_split': [2, 5, 10],     # Jumlah minimal sampel yang diperlukan untuk membagi node
    'min_samples_leaf': [1, 2, 4],        # Jumlah minimal sampel yang harus ada di node daun
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit grid search ke data training
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model terbaik
best_dt = grid_search.best_estimator_

# Buat prediksi dengan model terbaik
y_pred_best_dt = best_dt.predict(X_test_scaled)

# Evaluasi akurasi
acc_best_dt = accuracy_score(y_test, y_pred_best_dt)

# Print hasil
print("Test accuracy Decision Tree: {:.2f}".format(acc_best_dt))

Test accuracy Decision Tree: 1.00


Training dengan random forest

In [78]:
rf = RandomForestClassifier(n_estimators=10, random_state=42)

# Tentukan grid hyperparameter yang ingin dituning
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Jumlah pohon dalam hutan
    'max_depth': [None, 5, 10, 15, 20],   # Maksimal kedalaman pohon
    'min_samples_split': [2, 5, 10],      # Jumlah minimal sampel yang diperlukan untuk membagi node
    'min_samples_leaf': [1, 2, 4]         # Jumlah minimal sampel yang harus ada di node daun
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit grid search ke data training
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model terbaik
best_rf = grid_search.best_estimator_

# Buat prediksi dengan model terbaik
y_pred_best_rf = best_rf.predict(X_test_scaled)

# Evaluasi akurasi
acc_best_rf = accuracy_score(y_test, y_pred_best_rf)

# Print hasil
print("Test accuracy Random Forest: {:.2f}".format(acc_best_rf))

Test accuracy Random Forest: 1.00


# Tugas 2

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

training dengan decision tree

In [79]:
dt = DecisionTreeClassifier()

# Tentukan grid hyperparameter yang ingin dituning
param_grid = {
    'max_depth': [None, 5, 10, 15, 20],  # Maksimal kedalaman tree
    'min_samples_split': [2, 5, 10],     # Jumlah minimal sampel yang diperlukan untuk membagi node
    'min_samples_leaf': [1, 2, 4],        # Jumlah minimal sampel yang harus ada di node daun
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit grid search ke data training
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model terbaik
best_dt = grid_search.best_estimator_

# Buat prediksi dengan model terbaik
y_pred_best_dt = best_dt.predict(X_test_scaled)

# Evaluasi akurasi
acc_best_dt = accuracy_score(y_test, y_pred_best_dt)

# Print hasil
print("Test accuracy Decision Tree: {:.2f}".format(acc_best_dt))

Test accuracy Decision Tree: 1.00


trainiing dengan adaboost

In [80]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Inisialisasi model AdaBoost dengan algoritma SAMME
ada = AdaBoostClassifier(algorithm='SAMME')

# Tentukan grid hyperparameter yang ingin dituning
param_grid = {
    'n_estimators': [50, 100, 200],  # Jumlah estimators dalam AdaBoost
    'learning_rate': [0.01, 0.1, 1.0, 10.0],  # Laju pembelajaran
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=ada, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit grid search ke data training
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model terbaik
best_ada = grid_search.best_estimator_

# Buat prediksi dengan model terbaik
y_pred_best_ada = best_ada.predict(X_test_scaled)

# Evaluasi akurasi
acc_best_ada = accuracy_score(y_test, y_pred_best_ada)

# Print hasil
print("Test accuracy AdaBoost: {:.2f}".format(acc_best_ada))

Test accuracy AdaBoost: 1.00


Kesimpulan


Berdasarkan analisis perbandingan performa antara algoritma Decision Tree dan AdaBoost pada dataset mushroom, secara umum, AdaBoost cenderung memberikan akurasi yang lebih tinggi dibandingkan Decision Tree tunggal. Hal ini disebabkan oleh kemampuan AdaBoost dalam menggabungkan beberapa model lemah untuk membentuk model yang lebih kuat, sehingga lebih efektif dalam menangani kompleksitas

# Tugas 3

Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma

1. Logistic Regression

2. SVM kernel polynomial

3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

In [81]:
#impor library
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer


persiapan data

In [82]:
dbt = pd.read_csv('diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [83]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [84]:
# Cek kolom null
dbt.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [85]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [86]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.0,35.00000,155.548223,33.6,0.627,50.0,1
1,1.000000,85.0,66.0,29.00000,155.548223,26.6,0.351,31.0,0
2,8.000000,183.0,64.0,29.15342,155.548223,23.3,0.672,32.0,1
3,1.000000,89.0,66.0,23.00000,94.000000,28.1,0.167,21.0,0
4,4.494673,137.0,40.0,35.00000,168.000000,43.1,2.288,33.0,1


Split data training dan testing

In [87]:
# Split data training dan testing

X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

standarisasi fitur

In [88]:
from sklearn.preprocessing import StandardScaler

# Standarisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled

array([[-1.16422997, -0.89610788, -1.00440048, ..., -1.20403257,
        -0.61421636, -0.94861028],
       [ 0.22077929, -0.56399695, -0.02026586, ...,  0.66428525,
        -0.90973787, -0.43466673],
       [ 0.04580856,  0.43233584, -0.34831073, ...,  1.44035573,
        -0.30699103, -0.77729576],
       ...,
       [ 1.95204087, -0.69684133,  1.12789121, ...,  1.91462102,
         1.94892066,  0.42190587],
       [ 0.04580856,  0.63160239,  0.01296379, ...,  1.45472741,
        -0.77514391, -0.34900947],
       [ 0.04580856,  0.10022491,  1.9480034 , ..., -1.40523602,
        -0.60836445, -1.03426754]])

Training dengan logistic regression

In [89]:
# Inisialisasi model Logistic Regression
log_reg = LogisticRegression()

# Tentukan grid hyperparameter yang ingin dituning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Kekuatan regularisasi
    'solver': ['liblinear', 'lbfgs', 'saga'],  # Algoritma yang digunakan
    'max_iter': [100, 200, 300]  # Jumlah iterasi
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit grid search ke data training
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model terbaik
best_log_reg = grid_search.best_estimator_

# Buat prediksi dengan model terbaik
y_pred_best_log_reg = best_log_reg.predict(X_test_scaled)

# Evaluasi akurasi
acc_best_log_reg = accuracy_score(y_test, y_pred_best_log_reg)

# Print hasil
print("Test set accuracy: {:.2f}".format(acc_best_log_reg))
print(f"Test set accuracy: {acc_best_log_reg}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


Training dengan SVM Polynomial

In [90]:
# Tentukan parameter yang ingin dituning
param_grid = {
    'C': [0.1, 1, 10],                # Regularisasi parameter
    'degree': [2, 3, 4],              # Derajat polinomial
    'gamma': ['scale', 'auto']        # Kernel coefficient
}

# Inisialisasi SVM dengan kernel polinomial
svm_poly = SVC(kernel='poly')

# Lakukan grid search
grid_search = GridSearchCV(estimator=svm_poly, param_grid=param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model dengan parameter terbaik
best_svm_poly = grid_search.best_estimator_

# Prediksi menggunakan model terbaik
y_pred_svm_poly = best_svm_poly.predict(X_test_scaled)

# Evaluasi akurasi testing data
acc_svm_poly = accuracy_score(y_test, y_pred_svm_poly)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_poly))
print(f"Test set accuracy: {acc_svm_poly}")

Test set accuracy: 0.70
Test set accuracy: 0.696969696969697


Training dengan decision tree

In [91]:
# Tentukan parameter yang ingin dituning
param_grid = {
    'max_depth': [None, 5, 10, 15],    # Kedalaman maksimum dari pohon
    'min_samples_split': [2, 5, 10],   # Jumlah minimum sampel untuk membagi node
    'min_samples_leaf': [1, 2, 4]      # Jumlah minimum sampel yang harus ada di daun
}

# Inisialisasi Decision Tree
dt = DecisionTreeClassifier()

# Lakukan grid search
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Dapatkan model dengan parameter terbaik
best_dt = grid_search.best_estimator_

# Prediksi menggunakan model terbaik
y_pred_dt = best_dt.predict(X_test_scaled)

# Evaluasi akurasi testing data
acc_dt = accuracy_score(y_test, y_pred_dt)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.76
Test set accuracy: 0.7619047619047619


Ensemble voting

In [92]:
# Inisialisasi model
log_reg = LogisticRegression()
svm_poly = SVC(kernel='poly', probability=True)  # Menambahkan probability=True untuk voting
dt = DecisionTreeClassifier()

# Membuat model ensemble voting
voting = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('svm_poly', svm_poly),
    ('dt', dt)],
    voting='hard'  # Menggunakan voting keras
)

# Fit model voting ke data training
voting.fit(X_train_scaled, y_train)

# Buat prediksi dengan model voting
y_pred_voting = voting.predict(X_test_scaled)

# Evaluasi akurasi
acc_voting = accuracy_score(y_test, y_pred_voting)

# Print hasil
print("Test set accuracy Voting Classifier: {:.2f}".format(acc_voting))
print(f"Test set accuracy: {acc_voting}")

Test set accuracy Voting Classifier: 0.74
Test set accuracy: 0.7445887445887446
